In [2]:
import pandas as pd
import requests
import time
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# Kakao API 키 설정 (https://developers.kakao.com/ 에서 발급)
# 환경 변수에서 KAKAO_API_KEY 가져오기
KAKAO_API_KEY = os.getenv('KAKAO_API_KEY')

In [6]:
data = '../data/seoul_restaurant.csv'
df = pd.read_csv(data)
df.head()

,rest,category,score,count,latitude,longitude
0,할머니의 레시피 서울숲,한식,4.1,62,37.546169,127.043075
1,아이노가든키친 광화문,한식,4.5,28,37.572832,126.971270
2,도마 인사동,한식,4.4,66,37.573813,126.985835
3,별미볶음점 여의도,한식,4.5,38,37.520795,126.926931
4,녹원쌈밥 연희동,한식,4.3,26,37.572529,126.934160


In [8]:
def get_address_kakao(place_name):
    """Kakao Maps API로 장소명에 대한 도로명 주소 검색"""
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
    headers = {'Authorization': f'KakaoAK {KAKAO_API_KEY}'}
    params = {'query': place_name}

    try:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            result = response.json()
            if result['documents']:
                # 첫 번째 검색 결과의 도로명 주소 반환
                place = result['documents'][0]
                road_address = place.get('road_address_name', '')
                return road_address if road_address else place.get('address_name', '')
        time.sleep(0.1)  # API 호출 제한 방지
    except Exception as e:
        print(f"Error for {place_name}: {e}")

    return ''

# 도로명 주소 정보 가져오기
print("도로명 주소 정보를 가져오는 중...")
df['post'] = df['rest'].apply(get_address_kakao)

print(df[['rest', 'post']].head())

# 결과 저장
df.to_csv('../data/seoul_restaurant.csv', index=False, encoding='utf-8')
print(f"\n완료! 총 {len(df)}개 식당의 주소 정보를 추가했습니다.")
display(df.head())

도로명 주소 정보를 가져오는 중...
           rest                post
0  할머니의 레시피 서울숲  서울 성동구 서울숲2길 44-12
1   아이노가든키친 광화문     서울 종로구 경희궁2길 10
2        도마 인사동    서울 종로구 인사동8길 6-1
3     별미볶음점 여의도  서울 영등포구 국제금융로6길 33
4      녹원쌈밥 연희동   서울 서대문구 연희로25길 22

완료! 총 400개 식당의 주소 정보를 추가했습니다.


,rest,category,score,count,latitude,longitude,post
0,할머니의 레시피 서울숲,한식,4.1,62,37.546169,127.043075,서울 성동구 서울숲2길 44-12
1,아이노가든키친 광화문,한식,4.5,28,37.572832,126.971270,서울 종로구 경희궁2길 10
2,도마 인사동,한식,4.4,66,37.573813,126.985835,서울 종로구 인사동8길 6-1
3,별미볶음점 여의도,한식,4.5,38,37.520795,126.926931,서울 영등포구 국제금융로6길 33
4,녹원쌈밥 연희동,한식,4.3,26,37.572529,126.934160,서울 서대문구 연희로25길 22
